<a href="https://colab.research.google.com/github/vkrisvasan/100days_OSL/blob/main/2_OSL_pydantic_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install AI Suite
!pip install -qU pydantic-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.0/457.0 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.36.0 which is incompatible.


In [2]:

from pydantic_ai import Agent
from pydantic import BaseModel
import os
from google.colab import userdata
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [3]:
import os
import getpass
credential_names = ["GROQ_API_KEY","GEMINI_API_KEY",'WEATHER_API_KEY','GEO_API_KEY'] #,"OPENAI_API_KEY"]
for credential in credential_names:
  if credential not in os.environ:
    os.environ[credential]=getpass.getpass("Provide your..." + credential)

Provide your...GROQ_API_KEY··········
Provide your...GEMINI_API_KEY··········


In [4]:
#Define the Class
class CityLocation(BaseModel):
    city: str
    country: str

# Run the agent synchronously to conduct a conversation
agent = Agent('gemini-1.5-flash-8b', result_type=CityLocation)
result = agent.run_sync('Where the olympics held in 2012?')
print(result.data)
print(result.cost())

city='London' country='United Kingdom'
Cost(request_tokens=63, response_tokens=8, total_tokens=71, details=None)


In [6]:
import os
from pydantic import BaseModel
from pydantic_ai import Agent

# Define a Pydantic model for the expected output
class MyModel(BaseModel):
    city: str
    country: str

# Get the model name from environment variable or use a default
model_name = os.getenv('PYDANTIC_AI_MODEL', 'groq:llama-3.2-3b-preview')#'openai:gpt-4o')
print(f'Using model: {model_name}')

# Create the agent with the specified model and result type
agent = Agent(model_name, result_type=MyModel)


# Run the agent with a sample query
result = agent.run_sync('The windy city in the US of A.')

# Print the output data and cost
print(result.data)
print(f'Cost: {result.cost()}')

Using model: groq:llama-3.2-3b-preview


BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': 'geolocation_find_city>{"city": "Chicago", "country": "USA"}'}}

In [7]:
# Define a simple agent with a static system prompt
agent = Agent(
    model='groq:llama-3.2-3b-preview',#'openai:gpt-4o',
    system_prompt='Be concise, reply with one sentence.'
)

# Run the agent synchronously to conduct a conversation
result = agent.run_sync('Where does "hello world" come from?')
print(result.data)

The phrase "Hello, World!" originated from a 1978 book by Brian Kernighan and Dennis Ritchie, "The C Programming Language," which included a simple "Hello, World!" program as an example.


In [8]:
import os
import asyncio
from dataclasses import dataclass
from typing import Any, Dict

from httpx import AsyncClient
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext

# Define dependencies for the agent
@dataclass
class Deps:
    client: AsyncClient
    weather_api_key: str | None
    geo_api_key: str | None

# Create the weather agent
weather_agent = Agent(
    model='groq:llama-3.2-3b-preview',#'openai:gpt-4o',
    system_prompt='Be concise, reply with one sentence.',
    deps_type=Deps,
    retries=2,
)

@weather_agent.tool
async def get_lat_lng(ctx: RunContext[Deps], location: str) -> Dict[str, float]:
    """Get latitude and longitude for a given location."""
    if not ctx.deps.geo_api_key:
        return {'lat': 51.1, 'lng': -0.1}  # Dummy response for testing

    params = {'q': location, 'api_key': ctx.deps.geo_api_key}
    response = await ctx.deps.client.get('https://geocode.maps.co/search', params=params)
    response.raise_for_status()

    data = response.json()
    if data:
        return {'lat': data[0]['lat'], 'lng': data[0]['lon']}
    raise ValueError('Could not find the location')

@weather_agent.tool
async def get_weather(ctx: RunContext[Deps], lat: float, lng: float) -> Dict[str, Any]:
    """Get weather information for given latitude and longitude."""
    if not ctx.deps.weather_api_key:
        return {'temperature': '21 °C', 'description': 'Sunny'}  # Dummy response for testing

    params = {
        'apikey': ctx.deps.weather_api_key,
        'location': f'{lat},{lng}',
        'units': 'metric',
    }

    response = await ctx.deps.client.get('https://api.tomorrow.io/v4/weather/realtime', params=params)
    response.raise_for_status()

    values = response.json()['data']['values']

    # Weather code mapping
    code_lookup = {
        1000: 'Clear, Sunny',
        1001: 'Cloudy',
        4001: 'Rain',
        5000: 'Snow',
        # Add more codes as needed...
    }

    return {
        'temperature': f'{values["temperatureApparent"]:0.0f}°C',
        'description': code_lookup.get(values['weatherCode'], 'Unknown'),
    }

In [9]:
async def main():
    async with AsyncClient() as client:
        weather_api_key = userdata.get('WEATHER_API_KEY')
        geo_api_key = userdata.get('GEO_API_KEY')

        deps = Deps(client=client, weather_api_key=weather_api_key, geo_api_key=geo_api_key)

        result = await weather_agent.run('What is the weather like in Delhi?', deps=deps)
        print('Response:', result.data)

asyncio.run(main())

SecretNotFoundError: Secret WEATHER_API_KEY does not exist.

In [10]:
#Bank support agent
from dataclasses import dataclass
from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext

class FakeDatabase:
    """A mock database for demonstration purposes."""

    @classmethod
    async def get_customer_name(cls, id: int) -> str | None:
        return 'John' if id == 123 else None

    @classmethod
    async def get_customer_balance(cls, id: int) -> float:
        return 123.45 if id == 123 else None

@dataclass
class SupportDependencies:
    customer_id: int
    db: FakeDatabase

class SupportResult(BaseModel):
    support_advice: str = Field(description='Advice returned to the customer')
    block_card: bool = Field(description='Whether to block their card')
    risk: int = Field(description='Risk level of query', ge=0, le=10)

# Create the support agent
support_agent = Agent(
    model='openai:gpt-4o',
    deps_type=SupportDependencies,
    result_type=SupportResult,
    system_prompt=(
        'You are a bank support agent. Provide customer support '
        'and assess the risk level of their query. Use the customer\'s name in your response.'
    ),
)


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [11]:
# Example usage
deps = SupportDependencies(customer_id=123, db=FakeDatabase())
balance_result = support_agent.run_sync('What is my balance?', deps=deps)
print(balance_result.data)

loss_result = support_agent.run_sync('I just lost my card!', deps=deps)
print(loss_result.data)

NameError: name 'support_agent' is not defined